# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename
    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [4]:
infile = f"../data/sent_1874.txt"
sentences = MySentences(infile)

sentences = [sentence for sentence in sentences]

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [5]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [6]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [7]:
len(bigram_phrases.vocab.keys())

551708

Prenons une clé au hasard :

In [8]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

communale


Le dictionnaire indique le score de cette coocurrence :

In [19]:
bigram_phrases.vocab[key_]

1282

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

In [20]:
%time bigram_phrases[sentences[78]]

CPU times: user 193 µs, sys: 0 ns, total: 193 µs
Wall time: 225 µs


['ne_pouvait',
 'pas',
 'donner',
 'communication',
 'au',
 'public',
 'd',
 "'",
 'un',
 'proces_-',
 'verbal',
 'qui',
 'n',
 "'",
 'aurait',
 'pas',
 'encore',
 'ete',
 'a',
 'p',
 'p',
 'r',
 'o',
 'u',
 'v',
 'e',
 ';_mais',
 'on',
 'doit',
 'toujours']

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [9]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

In [10]:
%time bigram_phraser[sentences[78]]

CPU times: user 1.06 ms, sys: 346 µs, total: 1.4 ms
Wall time: 2.27 ms


[]

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [11]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [12]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [13]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [14]:
print(corpus[:100])

[['v', 'i', 'l', 'l', 'e', 'de'], [], ['bruxelles'], [], ['bulletin', 'ires', '8eanas'], ['dl', '!'], [], ['conseil'], [], ['communal'], [], ['annee'], [], ['1847', '.'], [], ['au'], [], ['ville', 'de', 'b', 'r', 'u', 'x', 'e', 'l', 'l', 'e', 's', '.'], [], ['bulletin'], [], ['conseil'], [], ['aes'], [], ['seances'], [], ['communal', '.'], [], ['annee'], [], ['1847', '.'], [], ['bruxelles', ','], ['imprimerie'], [], ['d', 'e', 'j', '.', 'h', '.', 'b', 'r', 'i', 'a', 'r', 'd', ','], [], ['rite', 'n', 'e', 'u', 'v', 'e', ',', '3', '1', ',', 'faubourg', 'de', 'n', 'a', 'm', 'u', 'r', ','], [], ['1', '84', '8'], [], ['de', '!'], [], ['du', 'consei'], ['dibi'], [], ['e', '.', '-', 'communication'], ['conclusions', 'de', 'la', 'section', 'des'], ['du', 'nouvel_hospice', 'pour', 'les', 'av'], [], ['enraisonde', 'l', "'", 'absence', '&'], ['maladie', '.', 'le', 'conseil', 'ajourne'], ['leurs', 'de', 'pierre', 'el', 'marchai'], ['cles', 'des', 'taxes', 'communale', "'"], ['bieniaance', 'eldesei

## Entrainement d'un modèle Word2Vec sur ce corpus

In [15]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 47.7 s, sys: 3.75 s, total: 51.4 s
Wall time: 32.1 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [16]:
outfile = f"../data/bulletins.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [17]:
model = Word2Vec.load("../data/bulletins.model")

### Imprimer le vecteur d'un terme

In [18]:
model.wv["bruxelles"]

array([ 1.5328453 ,  0.36960268,  0.00747779, -1.5477327 , -0.62328786,
       -0.24756345,  0.16112338, -0.6108562 , -1.0683087 , -2.749553  ,
        2.1452117 ,  1.3617662 ,  1.1070842 , -1.9560493 , -2.6213076 ,
       -1.2036617 , -0.712598  ,  1.9509077 , -2.6806483 , -1.3787805 ,
       -0.6620529 , -0.9177894 ,  3.9508474 , -1.7541779 , -1.0232117 ,
        2.3411438 , -1.752214  , -0.9388007 ,  0.7534014 ,  0.31978962,
        0.45079094, -0.29848757], dtype=float32)

### Calculer la similarité entre deux termes

In [19]:
model.wv.similarity("commune", "bruxelles")

0.5436324

### Chercher les mots les plus proches d'un terme donné

In [20]:
model.wv.most_similar("boucherie", topn=10)

[('partie_inferieure', 0.9281842112541199),
 ('partie_superieure', 0.9112018346786499),
 ('ruelle', 0.8946473598480225),
 ('fontaine', 0.8910585641860962),
 ('place_royale', 0.8830574750900269),
 ('sennette', 0.8763499855995178),
 ('digue', 0.8704918026924133),
 ('grille', 0.8672320246696472),
 ('colline', 0.8623616099357605),
 ('rue_leopold', 0.8621213436126709)]

### Faire des recherches complexes à travers l'espace vectoriel

In [21]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('paris', 0.6957014203071594), ('hotel', 0.6660506129264832), ('concert', 0.6480045914649963), ('liege', 0.6320095658302307), ('gand', 0.612479031085968), ('ten_-_noode', 0.605843186378479), ('londres', 0.6014090776443481), ('grand_etang', 0.6007885336875916), ('la_societe_civile', 0.5913822054862976), ('secours', 0.5898730754852295)]
